<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Eval_Cabrita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "22h/cabrita-lora-v0-1")

In [29]:
import torch
import time

def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Entrada:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:"""


# https://huggingface.co/docs/transformers/main_classes/text_generation
SAMPLE      = GenerationConfig(temperature=0.5, do_sample=True)
BEAM_SEARCH = GenerationConfig(temperature=0.5, num_beams=2)

def evaluate(instruction, gen_config=SAMPLE):
    prompt = generate_prompt(instruction)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    
    start = time.time()
    with torch.no_grad():
        generation_output = model.generate(
        input_ids=input_ids,
        generation_config=gen_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=128
    )
    
    time_elapsed = time.time() - start
    for s in generation_output.sequences:
        output = tokenizer.decode(s)

    output = output.split("### Resposta:")[1].strip()
    num_tokens = len(output.split())

    return output, num_tokens, time_elapsed

# ------------------------------------------------------------------------------------- # 
prompts = [
    "Você é um gerente do banco itaú. Escreva uma recomendação para um cliente investir em CDB.",
    """Escreva um texto para um cliente que possui gastos recorrentes no cartão com 
    despesa de PETs. O texto deve apresentar uma sugestão de contratação de assistência PET do banco itaú.""",
]

RUNS = 3
for run in range(RUNS):
    
    print(f'\n\nRUN: {run} --> BEAM-SEARCH:')
    for ix, p in enumerate(prompts):
        print(f'  ### Prompt: {p}')
        out, num_tokens, time_elapsed = evaluate("Instrução: " + p, gen_config=BEAM_SEARCH)
        print(f'  ### Resposta: {out}\n# qty ge tokens: {num_tokens} - time elapsed: {time_elapsed:.3} seg\n')
        print('\n', '-'*90)

    print(f'\n\nRUN: {run} --> SAMPLE:')
    for ix, p in enumerate(prompts):
        print(f'  ### Prompt: {p}')
        out, num_tokens, time_elapsed = evaluate("Instrução: " + p, gen_config=SAMPLE)
        print(f'  ### Resposta: {out}\n# qty ge tokens: {num_tokens} - time elapsed: {time_elapsed:.3} seg\n')

    print('='*110)



RUN: 0 --> BEAM-SEARCH:
  ### Prompt: Você é um gerente do banco itaú. Escreva uma recomendação para um cliente investir em CDB.
  ### Resposta: Você deve investir em Certificados de Depósito Bancários (CDBs) se você está procurando por um investimento seguro e confiável. Os CDBs oferecem uma taxa de juros mais alta do que os depósitos bancários tradicionais, mas com um risco muito menor. Eles também oferecem uma taxa de juros mais alta do que os depósitos bancários tradicionais, mas com um risco muito menor.
# qty ge tokens: 61 - time elapsed: 44.6 seg


 ------------------------------------------------------------------------------------------
  ### Prompt: Escreva um texto para um cliente que possui gastos recorrentes no cartão com 
    despesa de PETs. O texto deve apresentar uma sugestão de contratação de assistência PET do banco itaú.
  ### Resposta: "Gostaríamos de apresentar uma sugestão para você contratar assistência PET do Itaú. A assistência PET do Itaú pode ajudá-lo a co